In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import csv
import re

## HPG

In [3]:
def scp_hpg(area_cd,page_num):
    out = []
    for i in range(page_num-1):
        res = requests.get(f'https://www.hotpepper.jp/SA11/{area_cd}/lst/bgn{i+1}/')
        soup = BeautifulSoup(res.text, 'html.parser')
        page_lst = json.loads(soup.find("script",type="application/ld+json").contents[0])["itemListElement"]
        for page in page_lst:
            res = requests.get(page["url"])
            soup = BeautifulSoup(res.text, 'html.parser')
            try:
                name = soup.find("title").text.rstrip("＜ネット予約可＞ | ホットペッパーグルメ'").split("(")[0]
                area = soup.find("title").text.rstrip("＜ネット予約可＞ | ホットペッパーグルメ'").split("(")[1].split("/")[0]
                genre = soup.find("title").text.rstrip("＜ネット予約可＞ | ホットペッパーグルメ'").split("(")[1].split("/")[1].rstrip(")")
                tel = soup.find("span",class_="telNumber").text
            except:
                name = ""
                area = ""
                genre = ""
                tel = ""
            out.append([name,page["url"],area,genre,tel])
            time.sleep(2)
    return out

In [98]:
out_ueno = scp_hpg("Y015",218)
out_hachi = scp_hpg("Y110",185)

out_hpg = out_ueno + out_hachi
header = ["名前","URL","エリア","ジャンル","電話番号"]
with open('hpg.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(out_hpg)

## HPB 

In [2]:
def scp_hpb(area_cd,page_num,cat):
    out = []
    for i in range(page_num-1):
        res = requests.get(f'https://beauty.hotpepper.jp/{cat}/svcSA/{area_cd}/salon/PN{i+1}.html')
        soup = BeautifulSoup(res.text, 'html.parser')
        url_l = [soup.find_all("li",class_="searchListCassette")[i].find("h3",class_="slcHead").a.get('href').split("?")[0]+"tel/" for i in range(len(soup.find_all("li",class_="searchListCassette")))]
        
        for url in url_l:
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')   
            try:
                name = soup.title.text.rstrip("の電話番号｜ホットペッパービューティー")
                tel = soup.find("td",class_="fs16 b").text.rstrip()
            except:
                name = ""
                tel = ""
            out.append([name,url.rstrip("tel/"),tel])
            time.sleep(2)
    return out 

## エステ

In [100]:
out_ueno = scp_hpb("macAG",25,"esthe")
out_hachi = scp_hpb("macAK",25,"esthe")
out_esthe = out_ueno + out_hachi
header_esthe = ["名前","URL","電話番号"]
with open('hpb_esthe.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header_esthe)
    writer.writerows(out_esthe)

## ネイル

In [3]:
out_nakano = scp_hpb("macAI",10,"relax")
out_ueno = scp_hpb("macAG",31,"relax")
out_relax = out_ueno + out_nakano
header_nail = ["名前","URL","電話番号"]
with open('hpb_relax.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(header_nail)
    writer.writerows(out_relax)